In [43]:
import numpy as np, scipy as sp, pandas as pd, sklearn as sk
import time
from copy import deepcopy
from sklearn.neighbors import KNeighborsRegressor

In [23]:
small = pd.read_csv('small.csv') 
medium = pd.read_csv('medium.csv')
large = pd.read_csv('large.csv') 

In [29]:
def transition_calc(data):
    
    states, actions = data.s.unique(), data.a.unique()
    state_count, action_count = len(states), len(actions)
    
    tmat = np.zeros((state_count, action_count, state_count))
    rmat = np.zeros((state_count, action_count))

    for i in data.values:  
        tmat[i[0]-1, i[1]-1, i[3]-1] += 1
        rmat[i[0]-1, i[1]-1] = i[2]
    tmat /= np.sum(tmat, axis=-1, keepdims=True)
    
    return states, actions, tmat, rmat

In [45]:
def value_iteration(data, iters, discount, verbose):
    
    start = time.time()
    states, actions, tmat, rmat = transition_calc(data)
    state_count, action_count = len(states), len(actions)
    policy = np.zeros((iters, state_count))
    v = np.zeros(state_count)
        
    for i in range(iters):
        
        v_temp = np.zeros(state_count)
        
        for state in range(state_count):
            rewards = {}
            
            if verbose == True:
                print('\ncurrent state:', state, '\ncurrent value:', v[state])
                print('possible states:', np.where(tmat[state][0] !=0)[0],'\n')
            
            total = 0
            for action in range(action_count):
                exp_r = tmat[state][action]@rmat[:,action]
                possible_actions = np.where(tmat[state][action] != 0)[0]
                next_states = np.sum([v[j]*tmat[state][action][j] for j in possible_actions])
                rewards[action] = exp_r + (discount * next_states)

            if verbose == True:
                print('Actions/Rewards:\n', rewards)
            
            reward_max = max(rewards.values())
            v_temp[state] = reward_max

            best_action = [x==reward_max for x in rewards.values()].index(True)
            policy[i][state] = best_action
            
            if verbose == True:
                print('### Optimal Action: ', best_action,'###\n')

        v = v_temp

        
        if verbose == True:
            print('###############\nvalue update:', v, '\n###############')
    
    finish = time.time()
    print('Ran ', i, 'iterations in ', round(finish-start,2), ' seconds \n')

    return v, policy[-1]+1

In [14]:
def write_policy(dataset, policy):
    file = open(dataset + ".policy", "w")
    
    for i in range(len(policy)):
        state = str(int(policy[i])) +"\n"
        file.write(state)

    file.close()

In [15]:
def Q_update(data, Q, alpha, gamma):
    r = 0
    for i in range(len(data)):
        s, a, r, s_prime = data.values[i]
        Q[s-1, a-1] = Q[s-1, a-1] + alpha*(r + gamma*np.max(Q[s_prime - 1]) - Q[s-1, a-1])
    return Q

In [16]:
def Q_approx(data, Q, interval):
    m, n = Q.shape
    knn = KNeighborsRegressor(n_neighbors=interval)
    for i in range(n):
        for j in range(0, m, interval):
            x = np.matrix([[x for x in range(j,j+interval)],
                            np.tile(i, interval)]).reshape(interval,2)
            y = Q[j:j+interval,i]
            knn.fit(x, y) 
            for k in range(interval):
                if len(data[data.s==(j+k)])==0:
                    Q[j+k, i] = knn.predict([[j+k, i]])
    return Q

In [48]:
def Q_learn(data, states, actions, iters, alpha, gamma, interval, strategy):
    
    start = time.time()
    Q, Q_temp = np.zeros((states,actions)), np.zeros((states,actions))
    
    Q = Q_update(data, Q, alpha, gamma)
    if strategy == 'local':
        Q = Q_approx(data, Q, interval)  
    
    diff, i = 1, 0
    while diff > 0.1 and i < iters:
        Q = Q_update(data, Q, alpha, gamma)
        if strategy == 'local':
            Q = Q_approx(data, Q, interval)  
        
        diff = np.sum((Q_temp-Q)**2)
        i += 1
        Q_temp = deepcopy(Q)
        print('i: ',i,'diff: ',diff)
    iters = i
    
    policy = np.argmax(Q,axis=1)+1
    if strategy == 'random':
        for i in range(len(policy)):
            if policy[i] == 1:
                policy[i] = np.random.randint(actions) + 1
    
    finish = time.time()
    print('Ran ', iters, 'iterations in ', round(finish-start,2), ' seconds \n')
    
    return Q, policy

In [18]:
def randomizer(data, policy, actions):
    for i in range(len(policy)):
        if len(data[data.s==policy[i]])==0:
                policy[i] = np.random.randint(actions) + 1
    return policy

In [46]:
#small test

iters = 1000
discount = 0.95
verbose = False

s_v, s_policy = value_iteration(small, iters, discount, verbose)

Ran  999 iterations in  6.5  seconds 



In [47]:
#medium test

alpha = .95
gamma = 1

states = 50000
actions = 7

iters = 1000
interval = 10000
strategy = 'random'

m_Q, m_policy = Q_learn(medium, states, actions, iters, alpha, gamma, interval, strategy)
len(m_policy[m_policy!=1])

i:  1 diff:  7200568535662.95
i:  2 diff:  3754622361400.5767
i:  3 diff:  3702482538224.7847
i:  4 diff:  3613197869024.4043
i:  5 diff:  3626830389007.0005
i:  6 diff:  3694805493929.2705
i:  7 diff:  3828238853609.656
i:  8 diff:  3937880284753.306
i:  9 diff:  4175895346116.213
i:  10 diff:  4246574978128.268
i:  11 diff:  4168742936613.9473
i:  12 diff:  4023068727244.936
i:  13 diff:  3871206156675.785
i:  14 diff:  4105683599473.3105
i:  15 diff:  4326304488565.4194
i:  16 diff:  5128719863287.642
i:  17 diff:  7096407961528.852
i:  18 diff:  8306800970293.697
i:  19 diff:  7652034974343.873
i:  20 diff:  6264490394958.577
i:  21 diff:  5079499774524.687
i:  22 diff:  4276251762755.038
i:  23 diff:  3782343420211.319
i:  24 diff:  3459930283836.595
i:  25 diff:  3222027966279.9243
i:  26 diff:  3037323328944.2754
i:  27 diff:  2918182424958.4287
i:  28 diff:  2877952084322.7554
i:  29 diff:  3026472314692.5146
i:  30 diff:  3766183296604.9883
i:  31 diff:  5255999347108.431
i:  

45949

In [49]:
#large test

alpha = .95
gamma = .95

states = 312020
actions = 9

iters = 1000
interval = 78005
strategy = 'random'

l_Q, l_policy = Q_learn(large, states, actions, iters, alpha, gamma, interval, strategy)
len(l_policy[l_policy!=1])

i:  1 diff:  7627058397.763448
i:  2 diff:  31923601.36908219
i:  3 diff:  1332656.1376066145
i:  4 diff:  95405.35623918918
i:  5 diff:  8212.347932769057
i:  6 diff:  919.522093917451
i:  7 diff:  124.2274718237351
i:  8 diff:  18.18336598381174
i:  9 diff:  2.8821103106376293
i:  10 diff:  0.4657593804296159
i:  11 diff:  0.07628094372155905
Ran  11 iterations in  25.52  seconds 



277287

In [39]:
write_policy('small', s_policy)

In [40]:
write_policy('medium', m_policy)

In [41]:
write_policy('large', l_policy)